# Smart Wearables -- Bonsai Tree Classification on stream data

## 1. Introduction & Library Imports 

In [1]:
import numpy as np
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
from sklearn.model_selection import train_test_split
from bonsai.base.regtree import RegTree
from bonsai.base.alphatree import AlphaTree
from bonsai.base.c45tree import C45Tree
from bonsai.base.ginitree import GiniTree
from bonsai.base.xgbtree import XGBTree
from bonsai.base.friedmantree import FriedmanTree
from bonsai.ensemble.randomforests import RandomForests
from bonsai.ensemble.paloboost import PaloBoost
from bonsai.ensemble.gbm import GBM
import copy
import sys
import time

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
import math
from keras import optimizers
from utils import *
from model import *
from keras.utils.np_utils import to_categorical
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

#import pydot
#import graphviz
#pydot.find_graphviz = lambda: True
import keras

## 2. Data Gathering

In [2]:
data_input_file = 'data/LOTO/MHEALTH.npz'
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tmp = np.load(data_input_file)
np.load = np_load_old

In [3]:
X = tmp['X']
X = X[:, 0, :, :]
y = tmp['y']
folds = tmp['folds']

In [4]:
n_class = y.shape[1]
y = np.argmax(y, axis=1)
print('Hancrafted Template 2017 {}'.format(data_input_file))

Hancrafted Template 2017 data/LOTO/MHEALTH.npz


## 3. Feature Engineering

In [5]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

def feature_extraction(X):
    #Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(copy.copy(sample))
        features = np.hstack((features, A(copy.copy(sample))))
        features = np.hstack((features, SD(copy.copy(sample))))
        features = np.hstack((features, AAD(copy.copy(sample))))
        features = np.hstack((features, ARA(copy.copy(sample))))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X


## 4. RegTree

In [10]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
method
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = RegTree(max_depth=8)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.8516] Recall[0.8292] F1[0.8282] at fold[0]
______________________________________________________
Accuracy[0.9066] Recall[0.8922] F1[0.8952] at fold[1]
______________________________________________________
Accuracy[0.7537] Recall[0.7709] F1[0.7138] at fold[2]
______________________________________________________
Accuracy[0.7804] Recall[0.8017] F1[0.7484] at fold[3]
______________________________________________________
Accuracy[0.7757] Recall[0.7862] F1[0.7293] at fold[4]
______________________________________________________
Accuracy[0.8836] Recall[0.8821] F1[0.8572] at fold[5]
______________________________________________________
Accuracy[0.8962] Recall[0.9022] F1[0.8772] at fold[6]
______________________________________________________
Accuracy[0.9607] Recall[0.8949] F1[0.8970] at fold[7]
______________________________________________________
Accuracy[0.9500] Recall[0.9541] F1[0.9543] at fold[8]
______________________________________________________
Accuracy[0.8571] Re

In [11]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.8616] IC [0.8196, 0.9035]
Mean Recall[0.8564] IC [0.8224, 0.8905]
Mean F1[0.8292] IC [0.7823, 0.8762]
Mean size[0.867]
Mean training time[1155.292]
Mean prediction time[0.132]


## 5. XGBTree

In [12]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = XGBTree(max_depth=8,min_samples_split=1,min_samples_leaf=2)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.8711] Recall[0.8455] F1[0.8487] at fold[0]
______________________________________________________
Accuracy[0.8405] Recall[0.8384] F1[0.8177] at fold[1]
______________________________________________________
Accuracy[0.7649] Recall[0.7818] F1[0.7252] at fold[2]
______________________________________________________
Accuracy[0.8078] Recall[0.8233] F1[0.7687] at fold[3]
______________________________________________________
Accuracy[0.7757] Recall[0.7862] F1[0.7293] at fold[4]
______________________________________________________
Accuracy[0.9455] Recall[0.9424] F1[0.9454] at fold[5]
______________________________________________________
Accuracy[0.8846] Recall[0.8913] F1[0.8664] at fold[6]
______________________________________________________
Accuracy[0.9694] Recall[0.9664] F1[0.9375] at fold[7]
______________________________________________________
Accuracy[0.9333] Recall[0.9382] F1[0.8979] at fold[8]
______________________________________________________
Accuracy[0.8254] Re

In [13]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.8618] IC [0.8205, 0.9032]
Mean Recall[0.8647] IC [0.8264, 0.9030]
Mean F1[0.8316] IC [0.7848, 0.8784]
Mean size[0.640]
Mean training time[960.636]
Mean prediction time[0.164]


## 6. FriedmanTree

In [14]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = FriedmanTree(max_depth=10,min_samples_split=1,min_samples_leaf=1)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.8555] Recall[0.8312] F1[0.8326] at fold[0]
______________________________________________________
Accuracy[0.9494] Recall[0.9534] F1[0.9541] at fold[1]
______________________________________________________
Accuracy[0.6754] Recall[0.6952] F1[0.6091] at fold[2]
______________________________________________________
Accuracy[0.8039] Recall[0.8197] F1[0.7656] at fold[3]
______________________________________________________
Accuracy[0.7757] Recall[0.7862] F1[0.7293] at fold[4]
______________________________________________________
Accuracy[0.8909] Recall[0.8983] F1[0.8685] at fold[5]
______________________________________________________
Accuracy[0.8808] Recall[0.8891] F1[0.8640] at fold[6]
______________________________________________________
Accuracy[0.9738] Recall[0.9783] F1[0.9458] at fold[7]
______________________________________________________
Accuracy[0.9500] Recall[0.9541] F1[0.9307] at fold[8]
______________________________________________________
Accuracy[0.8056] Re

In [15]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.8561] IC [0.8023, 0.9099]
Mean Recall[0.8602] IC [0.8080, 0.9124]
Mean F1[0.8273] IC [0.7639, 0.8908]
Mean size[1.234]
Mean training time[1590.875]
Mean prediction time[0.137]


## 7. PaloBoost

In [16]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = PaloBoost(n_estimators=100,max_depth=5)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.8438] Recall[0.7948] F1[0.7756] at fold[0]
______________________________________________________
Accuracy[0.8560] Recall[0.8627] F1[0.8178] at fold[1]
______________________________________________________
Accuracy[0.8209] Recall[0.8332] F1[0.7941] at fold[2]
______________________________________________________
Accuracy[0.8588] Recall[0.8742] F1[0.8402] at fold[3]
______________________________________________________
Accuracy[0.8745] Recall[0.8804] F1[0.8639] at fold[4]
______________________________________________________
Accuracy[0.8400] Recall[0.8393] F1[0.7965] at fold[5]
______________________________________________________
Accuracy[0.9038] Recall[0.9094] F1[0.8970] at fold[6]
______________________________________________________
Accuracy[0.8996] Recall[0.8739] F1[0.8677] at fold[7]
______________________________________________________
Accuracy[0.8708] Recall[0.8335] F1[0.8134] at fold[8]
______________________________________________________
Accuracy[0.7659] Re

In [17]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.8534] IC [0.8302, 0.8766]
Mean Recall[0.8409] IC [0.8081, 0.8737]
Mean F1[0.8135] IC [0.7767, 0.8503]
Mean size[0.211]
Mean training time[22184.167]
Mean prediction time[7.539]


## 8. GBM

In [18]:
avg_acc = []
avg_recall = []
avg_f1 = []
avg_ttime=[]
avg_ptime=[]
avg_size=[]
for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

        #Your train goes here. For instance:
    #X_train=X_train.transpose(0,1,2).reshape(X_train.shape[0],-1)
    #X_test=X_test.transpose(0,1,2).reshape(X_test.shape[0],-1)
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)      
        
    method = GBM(n_estimators=100,max_depth=5)
    t0=time.time()
    method.fit(X_train, y_train)
    avg_ttime.append(time.time()-t0)
            #Your testing goes here. For instance:
    t1=time.time()
    y_pred = method.predict(X_test)
    avg_ptime.append(time.time()-t1)
    y_pred=np.round(y_pred,0)
    y_pred=y_pred.astype(int)
    v=method.dump()
    avg_size.append(round(v.__sizeof__()/1024,3))

    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc.append(acc_fold)

    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall.append(recall_fold)

    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i))
    print('______________________________________________________')

Accuracy[0.8516] Recall[0.8296] F1[0.8101] at fold[0]
______________________________________________________
Accuracy[0.8872] Recall[0.8954] F1[0.8705] at fold[1]
______________________________________________________
Accuracy[0.7948] Recall[0.8103] F1[0.7579] at fold[2]
______________________________________________________
Accuracy[0.8000] Recall[0.8181] F1[0.7750] at fold[3]
______________________________________________________
Accuracy[0.8859] Recall[0.8913] F1[0.8807] at fold[4]
______________________________________________________
Accuracy[0.9527] Recall[0.9681] F1[0.9588] at fold[5]
______________________________________________________
Accuracy[0.9346] Recall[0.9384] F1[0.9356] at fold[6]
______________________________________________________
Accuracy[0.9083] Recall[0.8901] F1[0.8887] at fold[7]
______________________________________________________
Accuracy[0.9417] Recall[0.8992] F1[0.8988] at fold[8]
______________________________________________________
Accuracy[0.6746] Re

In [19]:
ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))
print('Mean size[{:.3f}]'.format(np.mean(avg_size)))
print('Mean training time[{:.3f}]'.format(round(np.mean(avg_ttime)*1000,3)))
print('Mean prediction time[{:.3f}]'.format(round(np.mean(avg_ptime)*1000,3)))

Mean Accuracy[0.8631] IC [0.8132, 0.9131]
Mean Recall[0.8566] IC [0.8011, 0.9121]
Mean F1[0.8342] IC [0.7676, 0.9007]
Mean size[0.211]
Mean training time[85688.399]
Mean prediction time[8.123]


## 9. Conclusion

In [ ]:
from prettytable import PrettyTable    
x = PrettyTable()
x.field_names = ["Model", "Mean Accuracy", "Mean Recall", "Mean F1"]
x.add_row(["Reg Tree", 0.8616,0.8564,0.8292])
x.add_row(["XGB Tree", 0.8618, 0.8647, 0.8316])
x.add_row(["Friedman Tree", 0.8561, 0.8602, 0.8273])
x.add_row(["Palo Boost", 0.8534, 0.8409, 0.8135])
x.add_row(["GBM", 0.8631, 0.8566, 0.8342])

y = PrettyTable()
y.field_names = ["Model", "Mean Accuracy", "Mean Recall", "Mean F1"]
y.add_row(["Mean", 85.92, 0, 0])
print(x)
print(y)